# 第 7 章　函数装饰器和闭包

## 7.1　装饰器基础知识

装饰器是可调用的对象，其参数是另一个函数（被装饰的函数）

装饰器可能会处理被装饰的函数，然后把它返回，或者将其替换成另一个函数或可调用对象。

In [18]:
# 定义一个装饰器
def decorate(fun):
    print("decorate")
    return fun

# 使用装饰器
@decorate
def target():
    print('running target()')

print("decorator loaded")
target()
target()

decorate
decorator loaded
running target()
running target()


In [19]:
# 上面代码等价于下面的代码

def target():
    print('running target()')

target = decorate(target)

print("decorator loaded")
target()
target()

decorate
decorator loaded
running target()
running target()


严格来说，装饰器只是语法糖。

## 7.2 Python何时执行装饰器

装饰器的一个关键特性是，它们在被装饰的函数定义之后立即运行,且只运行一次。

## 7.3　使用装饰器改进“策略”模式

TODO

## 7.4　变量作用域规则

Python假定在函数定义体中赋值的变量是局部变量

在函数中直接给某个全局变量赋值会产生错误,
`UnboundLocalError: local variable 'a' referenced before assignment`
因为解释器在函数定义的时候把那些被赋值的变量当做局部变量，
如果非要这么做，要事先使用`global`声明

## 7.5 闭包

闭包指延伸了作用域的函数，其中包含函数定义体中引用、
但是不在定义体中定义的**非全局变量**。函数是不是匿名的没有关系，
关键是它能访问定义体之外定义的**非全局变量**。

In [20]:
# 计算移动平均值

def make_average():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

avg = make_average()

avg.__closure__

(<cell at 0x7fdbd2aec7f8: list object at 0x7fdbd2580808>,)

In [21]:
avg.__code__.co_freevars

('series',)

调用 `make_averager` 时，返回一个 `averager` 函数对象。
每次调用`averager` 时，它会把参数添加到系列值中，然后计算当前平均值

在 `averager` 函数中， `series` 是自由变量（free variable）。这是一
个技术术语，指未在本地作用域中绑定的变量。


## 7.6 nonlocal声明

使用闭包时，自由变量只能是可变类型的数据，不能是数字、字符串、元组等不可变类型，
否则只能读取不能修改，因为会遇到 7.4 中所说的问题,会隐式创建局部变量。

Python 3 引入了 nonlocal 声明。它的作用是主动把变量标记为自由变量,从而解决上述困扰。

## 7.7 实现一个简单的装饰器

一个简单的装饰器，输出函数的运行时间


In [22]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' %(elapsed, name, arg_str, result))
        return result

    return clocked

In [23]:
@clock
def snooze(seconds):
    time.sleep(seconds)

snooze(.123)

[0.12315090s] snooze(0.123) -> None


In [24]:
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

factorial(6)

[0.00000048s] factorial(1) -> 1
[0.00006903s] factorial(2) -> 2
[0.00015555s] factorial(3) -> 6
[0.00023894s] factorial(4) -> 24
[0.00032455s] factorial(5) -> 120
[0.00039273s] factorial(6) -> 720


720

In [25]:
# 若只想打印最后的运行时间

@clock
def factorial(n):
    def _f(n):
        return 1 if n < 2 else n * _f(n-1)
    return _f(n)

factorial(10)

[0.00000552s] factorial(10) -> 3628800


3628800

这是装饰器的典型行为：把被装饰的函数替换成新函数，二者接受相同的参数，
而且（通常）返回被装饰的函数本该返回的值，同时还会做些额外操作。

`clock`装饰器有几个缺点：不支持关键字参数，而且遮盖了被装饰函数的 `__name__` 和 `__doc__` 属性

## 7.8　标准库中的装饰器

Python 内置了三个用于装饰方法的函数： `property`、 `classmethod`和 `staticmethod`

另一个常见的装饰器是 `functools.wraps`，它的作用是协助构建行为良好的装饰器。

### 7.8.1　使用`functools.lru_cache`做备忘

`functools.lru_cache` 是非常实用的装饰器，它实现了备忘（memoization）功能。

对于斐波纳契数的计算，简单递归非常耗时

In [26]:
@clock
def fibonacci(n):
    def _f(n):
        return n if n < 2 else _f(n-2) + _f(n-1)
    return _f(n)
fibonacci(30)

[0.22742898s] fibonacci(30) -> 832040


832040

In [27]:
import functools
@clock
def fibonacci(n):
    @functools.lru_cache()
    def _f(n):
        return n if n < 2 else _f(n-2) + _f(n-1)
    return _f(n)
fibonacci(50)

[0.00003043s] fibonacci(50) -> 12586269025


12586269025

由此可见，`lru_cache`可以优化递归算法

### 7.8.2 单分派泛函数

TODO

## 7.9　叠放装饰器

把 `@d1` 和 `@d2` 两个装饰器按顺序应用到 `f` 函数上，作用相当于 `f = d1(d2(f))`


## 7.10　参数化装饰器

怎么让装饰器接受其他参数呢？答案是：
创建一个装饰器工厂函数，把参数传给它，返回一个装饰器，然后再把它应用到要装饰的函数上。




In [28]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry -> ', registry)
f1()

running register(<function f1 at 0x7fdbd228ca60>)
running main()
registry ->  [<function f1 at 0x7fdbd228ca60>]
running f1()


### 7.10.1　一个参数化的注册装饰器

为了便于启用或禁用 `register` 执行的函数注册功能，我们为它提供
一个可选的 `active` 参数，设为 False 时，不注册被装饰的函数。


In [29]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)

    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7fdbd228cae8>)
running register(active=True)->decorate(<function f2 at 0x7fdbd228c488>)


这里的关键是， `register()` 要返回 `decorate`，然后把它应用到装饰的函数上。

### 7.10.2　参数化clock装饰器

`clock` 装饰器添加一个功能：让用户传入一个格式字符串，控制被装饰函数的输出

In [30]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    # decorate才是装饰器
    def decorate(func):
        # 装饰器返回的函数
        def clocked(*_args):
            t0 = time.time()
            # 被装饰的函数在这里执行
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ','.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12316489s] snooze(0.123) -> None
[0.12316895s] snooze(0.123) -> None
[0.12314034s] snooze(0.123) -> None
